# Main Code

In [3]:
# Installing packages
#!pip install pandas openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


In [1]:
# Loading packages
import pandas as pd
import numpy as np

### Loading data files

In [52]:
# Loading data
data = pd.read_excel("Data/Data_final.xlsx") 

# Display the first 5 rows
print(data.head())

metadata = pd.read_excel("Data/Metadata_final.xlsx") 

# Display the first 5 rows  
print(metadata.head())

         Sample_Name  SeaDistance_m Location Duplicate    Datetime   Latitude  \
0  SLWT_2024_01_19_A            NaN    shore         A  2024_01_19  16.625000   
1  SLWT_2024_01_19_B            NaN    shore         B  2024_01_19  16.625000   
2  SLWT_2024_01_20_A            NaN    shore         A  2024_01_20  16.401667   
3  SLWT_2024_01_20_B            NaN    shore         B  2024_01_20  16.401667   
4  SLWT_2024_01_21_A            NaN    shore         A  2024_01_21  16.316667   

   Longitude  Type  HP_12CH4_dry_mean  HR_12CH4_dry_mean  ...  AirT_degC  \
0 -25.325000   NaN           1.854859           1.838359  ...       24.5   
1 -25.325000   NaN           1.729128           1.715480  ...       24.5   
2 -27.518333   NaN           1.827733           1.818951  ...       23.5   
3 -27.518333   NaN           1.892697           1.893364  ...       23.5   
4 -30.581667   NaN           1.902200           1.882740  ...       25.0   

   AirP_hPa  CH4atm_ppm  CO2atm_ppm  d13C_CH4atm_permil 

### Loading Variables

In [53]:
# Some usefull variables

R_JKmol = 8.314 # [J/K/mol], ideal gas law
V_w_L = 0.105 #[L], volume of the water sample in the syringe
V_hs_L = 0.35 #[L], volume of the headspace in the syringe
V_tot_L = 0.140 #[L], volume total of the syringe
P_atm_Pa = 101325 #[Pa], atmospheric pressure

### Calculation of ocean-air fluxes for CO2 and methane

Partial pressure - Pi [Pa]

In [54]:
# CO2
Pi_CO2_Pa = data["CO2atm_ppm"] * 10**(-6) * P_atm_Pa
#print(Pi_CO2_Pa)

#CH4
Pi_CH4_Pa = data["CH4atm_ppm"] * 10**(-6) * P_atm_Pa
#print(Pi_CH4_Pa)

Gas concentration in the water - n [mol]

In [55]:
#convert Temp at equilibrium (assumed to be the Temp after shaking) in Kelvin
Te_K = data["Te_degC"] + 273.15 
#print(Te_K)

In [56]:
# Ideal Gas Law (Magen et al., 2014) 
#converting V_w_L in m3
V_w_m3 = V_w_L / 1000

n_CO2_mol = Pi_CO2_Pa * V_w_m3 / (R_JKmol * Te_K)
#print(n_CO2_mol)

n_CH4_mol = Pi_CH4_Pa * V_w_m3 / (R_JKmol * Te_K)
#print(n_CH4_mol)

Corrected Henry's coefficients [mol/m3/Pa]

In [57]:
# CO2 (Weiss 1974)
A = [-58.0931, 90.5069, 22.2940]
B = [0.027766, -0.025888, 0.0050578]
hcpsalt_CO2_molm3Pa = (
    np.exp(
    A[0]
    + A[1] * 100 / Te_K
    + A[2] * np.log(Te_K / 100)
    + data["sal_psu"] * (B[0] + B[1] * Te_K / 100 + B[2] * (Te_K / 100) ** 2)
    ) * 1000/ 101325 
    )
#print(hcpsalt_CO2_molm3Pa)

In [58]:
# CH4 (Wiesenburg and Guinasso 1979)
A = [-417.5053, 599.8626, 380.3636, -62.0764]
B = [-0.064236, 0.03498, -0.0052732]
c_molm3 = (
    np.exp(
    + A[0]
    + A[1] * 100 / Te_K
    + A[2] * np.log(Te_K / 100)
    + A[3] * (Te_K / 100)
    + data["sal_psu"] * (B[0] + B[1] * Te_K / 100 + B[2] * (Te_K / 100) ** 2)
    ) * 1000 / 1e9
    )

hcpsalt_CH4_molm3Pa = c_molm3 / 101325

#print(hcpsalt__CH4_molm3Pa)

Dissolved water concentration [mol/m3] or solubility of the gases

In [59]:
# Henry’s Law (Sander, 2023)
# CO2
C_eq_CO2_molm3 = hcpsalt_CO2_molm3Pa * Pi_CO2_Pa
#print(C_eq_CO2_molm3)

# CH4
C_eq_CH4_molm3 = hcpsalt_CH4_molm3Pa * Pi_CH4_Pa
#print(C_eq_CH4_molm3)

Gas transfer velocity K600 [m/s]

In [60]:
# Function to extract min and max wind speed
kts_to_ms = 1852 / 3600

def WS_min_max(wind_speed):
    if isinstance(wind_speed, str) and '-' in wind_speed:  # Check if it's a range
        min_val, max_val = map(float, wind_speed.split('-'))  # Split and convert to float
    else:
        min_val = max_val = float(wind_speed)  # If it's a single number
    return pd.Series([min_val, max_val])

# Apply function to create new columns
metadata[['Min_Wind_Speed [kts]', 'Max_Wind_Speed [kts]']] = metadata['TWS [kts]'].apply(WS_min_max)

# Convert knots to meters per second
metadata['Min_Wind_Speed [m/s]'] = metadata['Min_Wind_Speed [kts]'] * kts_to_ms
metadata['Max_Wind_Speed [m/s]'] = metadata['Max_Wind_Speed [kts]'] * kts_to_ms

# Display the first rows
print(metadata.head())

         Sample_Name   Latitude  Longitude TWS [kts]  SOG [kts]  P_atm [hPa]  \
0  SLWT_2024_01_19_A  16.625000 -25.325000         7        4.5       1016.0   
1  SLWT_2024_01_19_B  16.625000 -25.325000         7        4.5       1016.0   
2  SLWT_2024_01_20_A  16.401667 -27.518333        16        7.6       1014.7   
3  SLWT_2024_01_20_B  16.401667 -27.518333        16        7.6       1014.7   
4  SLWT_2024_01_21_A  16.316667 -30.581667        19        7.5       1014.9   

   T_air [°C]   pH  Tw_b [°C]  Tw_a [°C] Mer [Douglas]  \
0        24.5  7.5       24.0       23.5             3   
1        24.5  7.5       23.5       23.5             3   
2        23.5  7.5       24.8       25.4           3-4   
3        23.5  7.5       24.5       25.2           3-4   
4        25.0  7.5       24.7       25.3             3   

                           Remarks  Min_Wind_Speed [kts]  \
0  Air pressure from NOAA archives                   7.0   
1  Air pressure from NOAA archives                

In [61]:
# Calculation of the gas transfer velocity K600 (MacIntyre et al., 2010) 
K600_min_ms = 2.04 * 10 * metadata["Min_Wind_Speed [m/s]"] + 2.0
#print(K600_min_ms)

K600_max_ms = 2.04 * 10 * metadata["Max_Wind_Speed [m/s]"] + 2.0
#print(K600_max_ms)

Schmidt number - Sc [-]

In [83]:
# (Wanninkhof, 2014) - temperature of the water in °C
#CO2
Sc_CO2 =  2073.1 - 125.62 * metadata["Tw_b [°C]"] + 3.6276 * metadata["Tw_b [°C]"]**2 - 0.043219 * metadata["Tw_b [°C]"]**3
#print(Sc_CO2)

#CH4
Sc_CH4 = 2039.2 - 120.31 * metadata["Tw_b [°C]"] + 3.4209 * metadata["Tw_b [°C]"]**2 - 0.040437 * metadata["Tw_b [°C]"]**3
#print(Sc_CH4)

Exchange coefficient - Ki [m/d]

In [63]:
# Define n from the gas exchange equation (n = 0.5 for wind speeds > 3.7 [m s-1] or, n = 0.66 for wind speeds < 3.7 [m s-1]) :
def WS_n (wind_speed) :
    if wind_speed < 3.7 :
        n = 0.66
    else:
        n = 0.5
    return n

metadata['Min_n'] = metadata['Min_Wind_Speed [m/s]'].apply(WS_n)
metadata['Max_n'] = metadata['Max_Wind_Speed [m/s]'].apply(WS_n)

print(metadata.head())

         Sample_Name   Latitude  Longitude TWS [kts]  SOG [kts]  P_atm [hPa]  \
0  SLWT_2024_01_19_A  16.625000 -25.325000         7        4.5       1016.0   
1  SLWT_2024_01_19_B  16.625000 -25.325000         7        4.5       1016.0   
2  SLWT_2024_01_20_A  16.401667 -27.518333        16        7.6       1014.7   
3  SLWT_2024_01_20_B  16.401667 -27.518333        16        7.6       1014.7   
4  SLWT_2024_01_21_A  16.316667 -30.581667        19        7.5       1014.9   

   T_air [°C]   pH  Tw_b [°C]  Tw_a [°C] Mer [Douglas]  \
0        24.5  7.5       24.0       23.5             3   
1        24.5  7.5       23.5       23.5             3   
2        23.5  7.5       24.8       25.4           3-4   
3        23.5  7.5       24.5       25.2           3-4   
4        25.0  7.5       24.7       25.3             3   

                           Remarks  Min_Wind_Speed [kts]  \
0  Air pressure from NOAA archives                   7.0   
1  Air pressure from NOAA archives                

In [64]:
# Find indices where Min and Max n are different
indices = metadata[metadata["Min_n"] != metadata["Max_n"]].index
print(indices)

Index([30, 31], dtype='int64')


In [68]:
# Gas exchange for CO2 (McGinnis et al., 2015) 
Ki_min_CO2_md = K600_min_ms * (600 / Sc_CO2)**(-metadata["Min_n"])
#print(Ki_min_CO2_md)

Ki_max_CO2_md = K600_max_ms * (600 / Sc_CO2)**(-metadata["Max_n"])
#print(Ki_max_CO2_md)

In [66]:
# Gas exchange for CH4 (McGinnis et al., 2015) 
Ki_min_CH4_md = K600_min_ms * (600 / Sc_CH4)**(-metadata["Min_n"])
#print(Ki_min_CH4_md)

Ki_max_CH4_md = K600_max_ms * (600 / Sc_CH4)**(-metadata["Max_n"])
#print(Ki_max_CH4_md)

Ocean-air fluxes - F [mmol/m2/d] 

In [80]:
#Fick’s first law for CO2 (Fick, 1855) 
F_min_CO2_mmolm2d = Ki_min_CO2_md * (data["co2_mmolm3"] - C_eq_CO2 * 10**3)
F_max_CO2_mmolm2d = Ki_max_CO2_md * (data["co2_mmolm3"] - C_eq_CO2 * 10**3)

df = pd.DataFrame({'Min CO2 flux': F_min_CO2_mmolm2d, 'Max CO2 flux': F_max_CO2_mmolm2d})
display(df)

#mean values
mean_F_min_CO2_mmolm2d = np.mean(F_min_CO2_mmolm2d)
mean_F_max_CO2_mmolm2d = np.mean(F_max_CO2_mmolm2d)
print("Mean min CO2 flux:", mean_F_min_CO2_mmolm2d)
print("Mean max CO2 flux:", mean_F_max_CO2_mmolm2d)

,Min CO2 flux,Max CO2 flux
0,302.317406,302.317406
1,552.171207,552.171207
2,573.398330,573.398330
3,595.466927,595.466927
4,52.638264,52.638264
5,143.268108,143.268108
6,245.246988,245.246988
7,204.061557,204.061557
8,154.897059,154.897059
9,64.520838,64.520838


Mean min CO2 flux: -113.00806101800265
Mean max CO2 flux: -125.45983758329314


In [95]:
#mean value of transatlantic samples
mean_min_F_CO2_transat = np.mean(F_min_CO2_mmolm2d[0:28])
mean_max_F_CO2_transat = np.mean(F_max_CO2_mmolm2d[0:28])
print("Transatlantic mean min CO2 flux :", mean_min_F_CO2_transat)
print("Transatlantic mean max CO2 flux :", mean_max_F_CO2_transat, "\n")

#mean value of Labrador sea samples
mean_min_F_CO2_labrador = np.mean(F_min_CO2_mmolm2d[28:38])
mean_max_F_CO2_labrador = np.mean(F_max_CO2_mmolm2d[28:38])
print("Labrador sea mean min CO2 flux :", mean_min_F_CO2_labrador)
print("Labrador sea mean max CO2 flux :", mean_max_F_CO2_labrador, "\n")

#mean value of Greeanland samples
mean_min_F_CO2_greenland = np.mean(F_min_CO2_mmolm2d[38:])
mean_max_F_CO2_greenland = np.mean(F_max_CO2_mmolm2d[38:])
print("Greenland mean min CO2 flux :", mean_min_F_CO2_greenland)
print("Greenland mean max CO2 flux :", mean_max_F_CO2_greenland)

Transatlantic mean min CO2 flux : 187.1603671717037
Transatlantic mean max CO2 flux : 199.82704662663167 

Labrador sea mean min CO2 flux : -859.5111191855494
Labrador sea mean max CO2 flux : -958.9413694902496 

Greenland mean min CO2 flux : -185.29155391310357
Greenland mean max CO2 flux : -184.93810210701076


In [81]:
#Fick’s first law for CH4 (Fick, 1855)
F_min_CH4_umolm2d = Ki_min_CH4_md * (data["ch4_mmolm3"] - C_eq_CH4 * 10**(3)) * 10**3
F_max_CH4_umolm2d = Ki_max_CH4_md * (data["ch4_mmolm3"] - C_eq_CH4 * 10**(3)) * 10**3

df = pd.DataFrame({'Min CH4 flux': F_min_CH4_umolm2d, 'Max CH4 flux': F_max_CH4_umolm2d})
display(df)

#mean values
mean_F_min_CH4_umolm2d = np.mean(F_min_CH4_umolm2d)
mean_F_max_CH4_umolm2d = np.mean(F_max_CH4_umolm2d)
print("Mean min CH4 flux:", mean_F_min_CH4_umolm2d)
print("Mean max CH4 flux:", mean_F_max_CH4_umolm2d)

,Min CH4 flux,Max CH4 flux
0,524.259220,524.259220
1,391.885438,391.885438
2,406.759620,406.759620
3,572.001057,572.001057
4,222.543107,222.543107
5,224.563880,224.563880
6,38.567153,38.567153
7,85.245167,85.245167
8,55.322180,55.322180
9,-85.376458,-85.376458


Mean min CH4 flux: 145.35375341259657
Mean max CH4 flux: 176.3381425681907


In [96]:
#mean value of transatlantic samples
mean_min_F_CH4_transat = np.mean(F_min_CH4_umolm2d[0:28])
mean_max_F_CH4_transat = np.mean(F_max_CH4_umolm2d[0:28])
print("Transatlantic mean min CO2 flux :", mean_min_F_CH4_transat)
print("Transatlantic mean max CO2 flux :", mean_max_F_CH4_transat, "\n")

#mean value of Labrador sea samples
mean_min_F_CH4_labrador = np.mean(F_min_CH4_umolm2d[28:38])
mean_max_F_CH4_labrador = np.mean(F_max_CH4_umolm2d[28:38])
print("Labrador sea mean min CO2 flux :", mean_min_F_CH4_labrador)
print("Labrador sea mean max CO2 flux :", mean_max_F_CH4_labrador, "\n")

#mean value of Greeanland samples
mean_min_F_CH4_greenland = np.mean(F_min_CH4_umolm2d[38:])
mean_max_F_CH4_greenland = np.mean(F_max_CH4_umolm2d[38:])
print("Greenland mean min CO2 flux :", mean_min_F_CH4_greenland)
print("Greenland mean max CO2 flux :", mean_max_F_CH4_greenland)

Transatlantic mean min CO2 flux : 106.45073947935973
Transatlantic mean max CO2 flux : 115.84364820346092 

Labrador sea mean min CO2 flux : 272.6311493473484
Labrador sea mean max CO2 flux : 402.1032059867356 

Greenland mean min CO2 flux : 131.2391711651437
Greenland mean max CO2 flux : 132.96854318565101


In [104]:
# Creating a big datafrane with all the results
# for CO2
df = pd.DataFrame({'Pi CO2 [Pa]' : Pi_CO2_Pa, 'n CO2 [mol]' : n_CO2_mol, 'Henrys coeff [molm3Pa] ' : hcpsalt_CO2_molm3Pa, 
                   'C_eq CO2 [molm3]' : C_eq_CO2_molm3, 'Min K600 [m/s]' : K600_min_ms, 'Max K600 [m/s]' : K600_max_ms,
                   'Sc CO2 [-]' : Sc_CO2, 'Min Ki CO2 [m/d]' : Ki_min_CO2_md, 'Max Ki CO2 [m/d]' : Ki_max_CO2_md,  
                   'Min CO2 flux [mmol/m2/d]': F_min_CO2_mmolm2d, 'Max CO2 flux [mmol/m2/d]': F_max_CO2_mmolm2d})
display(df)

,Pi CO2 [Pa],n CO2 [mol],Henrys coeff [molm3Pa],C_eq CO2 [molm3],Min K600 [m/s],Max K600 [m/s],Sc CO2 [-],Min Ki CO2 [m/d],Max Ki CO2 [m/d],Min CO2 flux [mmol/m2/d],Max CO2 flux [mmol/m2/d]
0,43.081744,0.000002,0.000296,0.012769,75.462667,75.462667,550.258144,71.273190,71.273190,302.317406,302.317406
1,43.081744,0.000002,0.000296,0.012769,75.462667,75.462667,563.481320,72.399040,72.399040,552.171207,552.171207
2,38.189018,0.000002,0.000282,0.010776,169.914667,169.914667,529.624043,159.639033,159.639033,573.398330,573.398330
3,38.189018,0.000002,0.000284,0.010831,169.914667,169.914667,537.292884,160.790649,160.790649,595.466927,595.466927
4,45.106187,0.000002,0.000283,0.012757,201.398667,201.398667,532.171735,189.673592,189.673592,52.638264,52.638264
5,45.106187,0.000002,0.000287,0.012954,201.398667,201.398667,537.292884,190.584032,190.584032,143.268108,143.268108
6,43.956948,0.000002,0.000256,0.011249,132.133867,132.133867,499.620456,120.575376,120.575376,245.246988,245.246988
7,43.956948,0.000002,0.000265,0.011630,132.133867,132.133867,499.620456,120.575376,120.575376,204.061557,204.061557
8,44.130671,0.000002,0.000266,0.011735,148.925333,148.925333,489.805560,134.556540,134.556540,154.897059,154.897059
9,44.130671,0.000002,0.000265,0.011679,148.925333,148.925333,487.362717,134.220579,134.220579,64.520838,64.520838


In [103]:
# for CH4
df = pd.DataFrame({'Pi CH4 [Pa]' : Pi_CH4_Pa, 'n CH4 [mol]' : n_CH4_mol, 'Henrys coeff CH4 [molm3Pa] ' : hcpsalt_CH4_molm3Pa, 
                   'C_eq CH4 [molm3]' : C_eq_CH4_molm3, 'Min K600 [m/s]' : K600_min_ms, 'Max K600 [m/s]' : K600_max_ms,
                   'Sc CH4 [-]' : Sc_CH4, 'Min Ki CH4 [m/d]' : Ki_min_CH4_md, 'Max Ki CH4 [m/d]' : Ki_max_CH4_md,  
                   'Min CH4 flux': F_min_CH4_umolm2d, 'Max CH4 flux': F_max_CH4_umolm2d})
display(df)

,Pi CH4 [Pa],n CH4 [mol],Henrys coeff CH4 [molm3Pa],C_eq CH4 [molm3],Min K600 [m/s],Max K600 [m/s],Sc CH4 [-],Min Ki CH4 [m/d],Max Ki CH4 [m/d],Min CH4 flux,Max CH4 flux
0,0.143555,6.111594e-09,0.000011,0.000002,75.462667,75.462667,563.197312,72.374954,72.374954,524.259220,524.259220
1,0.143555,6.111594e-09,0.000011,0.000002,75.462667,75.462667,576.320694,73.483647,73.483647,391.885438,391.885438
2,0.171231,7.243424e-09,0.000010,0.000002,169.914667,169.914667,542.717098,161.600238,161.600238,406.759620,406.759620
3,0.171231,7.248280e-09,0.000010,0.000002,169.914667,169.914667,550.328648,162.729505,162.729505,572.001057,572.001057
4,0.189187,8.005712e-09,0.000010,0.000002,201.398667,201.398667,545.245711,191.989331,191.989331,222.543107,222.543107
5,0.189187,8.021839e-09,0.000011,0.000002,201.398667,201.398667,550.328648,192.882145,192.882145,224.563880,224.563880
6,0.189799,7.922750e-09,0.000010,0.000002,132.133867,132.133867,512.947688,122.172949,122.172949,38.567153,38.567153
7,0.189799,7.959581e-09,0.000010,0.000002,132.133867,132.133867,512.947688,122.172949,122.172949,85.245167,85.245167
8,0.190664,8.001154e-09,0.000010,0.000002,148.925333,148.925333,503.216016,136.386123,136.386123,55.322180,55.322180
9,0.190664,7.995840e-09,0.000010,0.000002,148.925333,148.925333,500.794619,136.057593,136.057593,-85.376458,-85.376458
